In [23]:

import pandas as pd
from fastai.tabular.all import *
import torch.nn as nn
import torch.nn.functional as F
from torch import tensor
import torch.optim as optim

pd.options.display.float_format = '{:2.f}'.format
set_seed(12)
df = pd.read_csv("train.csv")
GET_ROW = 0
EEG_PATH = 'train_eegs/'
SPEC_PATH = 'train_spectrograms/'
n_coeff = 50 * 20 * 200
t_dep_list = []
t_indep_list = []
train = pd.read_csv('train.csv')
for ROW in range(200):
    row = train.iloc[ROW]

    t_dep_list.append([1 if x == max(row[9:15]) else 0 for x in row[9:15]])
    #t_dep_list.append(row[9:15])
    eeg = pd.read_parquet(f'{EEG_PATH}{row.eeg_id}.parquet')
    eeg_offset = int( row.eeg_label_offset_seconds )
    eeg = eeg.iloc[eeg_offset*200:(eeg_offset+50)*200]
    flat = eeg.values.flatten()
    max_val = max(eeg.values.flatten())
    min_val = min(eeg.values.flatten())
    scaled = [x/(max_val-min_val)  for x in eeg.values.flatten()]
    t_indep_list.append(scaled)



t_dep = tensor(t_dep_list, dtype=torch.long) 
t_indep = tensor(t_indep_list)
print(t_dep)
print(t_indep)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(n_coeff, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 6)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x)

net = Net()
print(net)


loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


tensor([[1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1]])
tensor([[-0.0441, -0.0386, -0.0439,  ..., -0.0593, -0.0626,  0.0140],
        [-0.0709, -0.0633, -0.0631,  ..., -0.0947, -0.0731,  0.0560],
        [-0.0430, -0.0500, -0.0525,  ..., -0.1039, -0.0763,  0.1030],
        ...,
        [-0.4271, -0.2522, -0.2110,  ..., -0.2196,  0.0566, -0.0720],
        [-0.4923, -0.2436, -0.1287,  ..., -0.1767,  0.0600, -0.0995],
        [-0.2791, -0.1150, -0.0844,  ..., -0.1120,  0.1656, -0.0813]])
Net(
  (fc1): Linear(in_features=200000, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=6, bias=True)
)


In [27]:
t_dep[31]

tensor([1, 0, 0, 0, 0, 0])

In [9]:
F.log_softmax(tensor([0,1,1,1]), dtype=float)

/tmp/ipykernel_27796/4179861982.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  F.log_softmax(tensor([0,1,1,1]), dtype=float)


tensor([-2.2143, -1.2143, -1.2143, -1.2143], dtype=torch.float64)

In [28]:
for epoch in range(3):
    for i in range(200):
        X = t_indep[i]
        y = t_dep[i]
        net.zero_grad()
        output = net(X)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print(loss)

/tmp/ipykernel_27796/1238573712.py:52: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


: 

In [22]:
net(t_indep[14])

tensor([0, 0, 0, 0, 0, 0])